##  Flow

![img](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Images/pasted-image.png)

##  About

| Property             | Details                                    |
|---------------------|-------------------------------------------|
| **Ticker/Instrument** | XAUUSD                                     |
| **Broker**            | OANDA (TradingView only)                  |
| **Timeframe**         | Non-specific                               |
| **Timezone**          | Asia/Manila (UTC+8)                        |
| **Session**           | New York (customed 1h)                    |
| **Method**            | Range Breakout                             |

##  Details

Locate the x-y coordinate of candles formed from 8pm to 9pm (20 oclock in asianmanila timeframe).

Among these candles, find the highest point and lowest point, and make vertical lines until it makes a rectangle from those vertices. This will serve as range.
   
    ┌──────────────  sessionHigh
    │
    │   ████  ← 8PM–9PM candles
    │
    └──────────────  sessionLow

Locate the y-coordinate and mark the high, low, and midpoit of the range.

    ┌────────────── HIGH   (green)
    │   ████
    │   ████
     ────────────── MID    (orange)
    │   ████
    │   ████
    └────────────── LOW    (red)

The high will serve as buystop and to calculate/locate it's TP y-coordinate, absolute value of high minus mid and times it by two, then add it to the high(buystop price) to project the y-coordinate of it's TP. 

Inverse the logic or mirror it for the sellstop.

The midpoint will serve as SL for both orders.

**One range one setup two orders four outcomes** 

    ─────────────── TP
    
    
    
     
     
    ┌─────────────► buystop
    │    ████    │
    │    ████    │
     ────────────── SL
    │    ████    │
    │    ████    │
    └─────────────► sellstop
    
       
    
    
    
    ─────────────── TP

locate and highlight the candle at 4am. This will serve as the validation for the setup.

##  **Setup Overview:**  
This framework defines the conditions for executing the trading strategy.

![Template](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Pinescript%20Language/Strategies/images/pasted-image.png)

---

## Rules

1. Use **Asia/Manila timezone (UTC+8)**.  
2. Identify the **8 PM – 9 PM price range**.  
3. Candles outside this range will guide trading decisions.  
4. The developing price (including wicks) **must revisit the mid-price** of the range before proceeding.  
5. Once the mid-price is revisited:  
   - Place a **Buy Stop** at the range **high**.  
   - Place a **Sell Stop** at the range **low**.  
6. **Take Profit (TP):** 2R multiple from the entry.  
7. **Stop Loss (SL):** mid-price for both Buy Stop and Sell Stop.  
8. Both Buy Stop and Sell Stop **may trigger** in the same setup, valid until the **4 AM candle**.  
9. Maximum outcomes per setup: **2 wins, 2 losses, or a mix**.  
10. If the trade hasn’t hit any target by 4 AM, **cancel the orders**.  

## Risk Management

- Starting capital: **$2,000**  
- Risk per trade: **3% of current capital** (applies separately to Buy Stop and Sell Stop).  

## Implementation Note

Use the **y-coordinates from the indicator** to calculate the range, entries, SL, and TP targets.

![Strategy Example](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Pinescript%20Language/Strategies/images/Screenshot%202025-11-12%20201257.png)

---

##  Result

![Test 1 Result](https://github.com/algorembrant/Pinescript.TradingView-Indicators.and.Strategies/blob/main/Pinescript%20Language/Strategies/images/Screenshot%202025-11-12%20205738.png)"